In [373]:
import pandas as pd

ROOT = "/Users/rafaelfrade/arquivos/desenv/lse/anc_hiv_scheduling/data/"
baseline_path=f"{ROOT}/hiv/baseilne/aws_files"
cleaned_data=f"{ROOT}/cleaned_data"

import pandas as pd
import numpy as np
import json
# package to read aws json files
from trp import Document
from os import listdir

In [361]:
def get_facility_page(file_name):
    """
        extract facility and day  from
        filename
    """
    file_name = file_name.replace(".txt", "")
    facility = int(file_name.split("_")[1].replace("US", ""))
    page = int(file_name.split("_")[2]
                    .replace("page", ""))
    
    return facility, page

assert get_facility_page("baseline_US1_page1.txt") == (1,1)

def remove_special_characters(string):
    """
        removes ":.,\ ", leading "0" to compare to annotated data
    """
    return (string
             .replace(":", "")
             .replace(".", "")
             .replace(" ", "")
             .replace("\'", "") # Attention: removing "'", not \
             .replace("\\", "")
             .replace("\/", "")
             .replace(",", "")
             .replace("i", "")
             .replace("-", ""))

In [87]:
def get_dictionary_response():
    total = 0

    dict_response = {"file_name":[], "facility":[],
                     "day": [], "line":[],
                     "waiting_time":[]}
    responses_sample = [f for f in listdir(baseline_path) if "txt" in f]
    for file in responses_sample:
        total += 1
        if total % 100 == 0:
            print(total)

        f = open(f"{baseline_path}/{file}", "r")
        response_string = f.read()
        response_json = json.loads(response_string)
        doc = Document(response_json)
        doc_page = doc.pages[0]

        if len(doc_page.tables) == 0:
            continue
        table = doc_page.tables[0]

        first_line_cells = table.rows[0].cells
        index_waiting_time = 0 # 
        for cell in first_line_cells:
            c = cell.text
            if "Tempo" in c or "fila" in c or "aguardou" in c:
                break
            index_waiting_time += 1

        line_number = 1
        # skip first because it's the header
        for row in table.rows[1:len(table.rows)]:

            facility, page = get_facility_page(file)
            waiting_time = row.cells[index_waiting_time].text

            dict_response["file_name"].append(file)
            dict_response["facility"].append(facility)
            dict_response["page"].append(page)
            dict_response["line"].append(line_number)
            dict_response["waiting_time"].append(waiting_time)

            line_number += 1


100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300


In [92]:
pd.set_option('display.max_rows', 6000)

In [367]:
replace_dic_1st = {
    "mim":"min",
    "mm":"min",
    "NOT_SELECTED":"",
    "Bom":"30m",
    "Bm":"3m",
    "mi-":"mi",
    "+/-":"",
}
 
case_sensitive_replace = {
    "amin":"9min",
    "Amin":"1min",
    "ZMIN":"1min"
}

replace_dic_sec = {
    "sm":"5m",
    "smin":"5min",
    "jomin":"20min",
    "zomin":"20min",
    "domin":"20min",
    "lomin":"10min",
    "romin":"10min",
    "tomin":"10min",
    "tom":"10m",
    "th":"1h",
    "ih":"1h",
    "zh":"2h",
    "lom":"10m",
    "uh":"4h",
    "2n":"2h",
    "lh":"1h",
    "Amin":"1min",
    "us":"45",
    "gmin":"9min",
    "jh":"2h",
    "bm":"6m",
    "hon":"hor",
    "selected":"",
    ":h":"h",
    "1e":"1h",
    "2e":"2h",
    "3e":"3h"
}

final = {
    "omin":"0min",
    "z":"2"
}

exact_replace = {
    "the":"1h",
    "14":"1h",
    "gh":"9h",
    "hora":"1h",
    "sh":"5h"
}

contains_replace_all = {
    "1n":"1h",
    "lh":"1h",
    "seg":"0"
}

starts_with = {
    "is":"15",
    "as":"15",
    "sh":"5h",
}

In [362]:
def replace_with_dic(time, dictionary):
    for from_,to_ in dictionary.items():
        time = time.replace(from_, to_)
    return time

assert replace_with_dic("amin", case_sensitive_replace) == "9min"

def replace_patterns(wt):
    wt = replace_with_dic(wt, replace_dic_1st)
    wt = replace_with_dic(wt, case_sensitive_replace)
    
    wt = wt.lower()
    wt = wt.replace(" ", "")
    wt = replace_with_dic(wt, replace_dic_sec)
    
    for from_, to_ in contains_replace_all.items():
        if from_ in wt:
            return to_
    
    wt = remove_special_characters(wt)
    
    for from_, to_ in exact_replace.items():
        if from_ == wt:
            return to_

    for from_, to_ in starts_with.items():
        if wt.startswith(from_):
            wt = wt.replace(from_, to_)
    
    return wt

In [375]:
def generate_hiv_baseline():
    hiv = pd.DataFrame(get_dictionary_response())
    hiv["hour"] = np.nan
    hiv["minute"] = np.nan

    hiv = hiv.query("~ waiting_time.str.contains('aguardou')")
    hiv = hiv.query("~ waiting_time.str.contains('control')")
    hiv["wt_replaced"] = hiv["waiting_time"].apply(replace_patterns)

    # 1st attempt to find final data: regex
    hiv["hour"] = hiv["wt_replaced"].apply(find_regex).apply(lambda x: x[0])
    hiv["minute"] = hiv["wt_replaced"].apply(find_regex).apply(lambda x: x[1])

    # 2nd attempt to find final data: find hour
    hiv.loc[hiv["hour"].isna(), "hour"] = hiv.loc[hiv["hour"].isna(), "wt_replaced"].apply(extract_time).apply(lambda x: x[0])
    hiv.loc[hiv["minute"].isna(), "minute"] = hiv.loc[hiv["minute"].isna(), "wt_replaced"].apply(extract_time).apply(lambda x: x[1])

    hiv.loc[hiv.eval("hour > 9"), "hour"] = np.nan
    hiv.loc[hiv.eval("minute > 100"), "minute"] = np.nan

    hiv["waiting_time_minutes"] = hiv["hour"]*60 + hiv["minute"]

    # remove empty lines
    hiv = hiv.query("waiting_time != '' ")
    hiv.to_csv(f"{cleaned_data}/hiv_baseline.csv", index=False, mode="w")

In [382]:
hiv.shape

(20381, 9)

In [383]:
13.8 / 18.3

0.7540983606557378

In [352]:
def extract_minute(time):
    if "m" in time:
        before_min = time.split("m")[0]
        if before_min.isnumeric():
            return float(before_min)
    
    return None

assert extract_minute("30min") == 30

def extract_time(time):
    hour = np.nan
    minute = np.nan
    if "h" in time:
        before_h = time.split("h")[0]
        if before_h.isnumeric():
            hour = float(before_h)
            before_min = extract_minute(time.split("h")[1])
            if before_min:
                minute = float(before_min)
            else:
                minute = 0
    else:
        before_min = extract_minute(time)
        if before_min:
            hour = 0
            minute = float(before_min)
    return hour, minute

assert extract_time("1h30min") == (1.0, 30.0)
assert extract_time("30min") == (0, 30.0)


In [369]:
hiv.query("minute.isna()")["wt_replaced"].value_counts()

wt_replaced
                   2034
n                   275
mn                  267
30                   69
h                    63
((((                 56
mnutes               55
20                   52
(                    46
u5mn                 38
((                   32
somn                 31
5                    30
(((                  27
0                    26
10                   26
ah                   24
zom                  20
omn                  19
40                   17
quatramnuntos        16
0030                 15
da5mnuntos           15
mnute                14
mnuto                14
50                   14
oh                   14
om                   13
3                    13
15                   12
for                  12
lmn                  12
zmw                  12
an                   12
cncamnuntos          11
tm                   10
60                   10
0010                 10
fom                  10
mnut                 10
so                   10
zmn 

In [359]:
hiv.query("waiting_time.notna()").sample(5000)

,file_name,facility,day,line,waiting_time,wt_replaced,hour,minute
8271,baseline_US46_page17.txt,46,17,5,22 Minute,22minute,0.0,22.0
18119,baseline_US78_page17.txt,78,17,5,33 Me,33me,0.0,33.0
19268,baseline_US16_page45.txt,16,45,5,25 minutos,25minutos,0.0,25.0
18165,baseline_US19_page1.txt,19,1,3,00:30,00:30,0.0,30.0
685,baseline_US23_page14.txt,23,14,5,uh,4h,4.0,0.0
17947,baseline_US14_page6.txt,14,6,3,Sminutos,5minutos,0.0,5.0
13238,baseline_US13_page36.txt,13,36,1,14,1h,1.0,0.0
19745,baseline_US53_page15.txt,53,15,6,"NOT_SELECTED, 2h",",2h",NaN,NaN
13901,baseline_US32_page5.txt,32,5,4,30m,30m,0.0,30.0
9863,baseline_US78_page1.txt,78,1,4,5.15m,5.15m,NaN,NaN


In [348]:

import re
def find_regex(time):
    """
        finds patterns D:D and DhD
        return hour and minutes
    """
    
    regex_2dots = re.compile(r'\d\:\d', re.I)
    regex_h = re.compile(r'\dh\d', re.I)
    #regex_dot = re.compile(r'\d\.\d', re.I)

    hour = np.nan
    minute = np.nan

    m = regex_2dots.search(time)
    if m:
        pattern = m.group()
        hour = pattern.split(":")[0]
        minute = pattern.split(":")[1]

    m = regex_h.search(time)
    if m:
        pattern = m.group()
        hour = pattern.split("h")[0]
        minute = pattern.split("h")[1]
    return float(hour), float(minute)*10


In [345]:
hiv.query("waiting_time.str.contains('\d\.\d')")

,file_name,facility,day,line,waiting_time,wt_replaced,hour,minute
50,baseline_US78_page24.txt,78,24,6,4.9g,4.9g,NaN,NaN
478,baseline_US64_page31.txt,64,31,4,8.00,8.00,NaN,NaN
782,baseline_US64_page18.txt,64,18,5,4.21 a,4.21a,NaN,NaN
791,baseline_US64_page30.txt,64,30,2,1.4,1.4,NaN,NaN
1017,baseline_US24_page2.txt,24,2,2,1.4,1.4,NaN,NaN
1820,baseline_US64_page26.txt,64,26,2,2.14,2.14,NaN,NaN
1878,baseline_US10_page80.txt,10,80,3,2.9mm,2.9min,NaN,NaN
2342,baseline_US22_page53.txt,22,53,6,1.30h,1.30h,NaN,NaN
3982,baseline_US1_page63.txt,1,63,6,2.430.00,2.430.00,NaN,NaN
4705,baseline_US10_page78.txt,10,78,1,4.5mm,4.5min,NaN,NaN


In [267]:
hiv.query("waiting_time.str.contains(':', regex=True)")

,file_name,facility,day,line,waiting_time,wt_replaced,hour
55,baseline_US1_page66.txt,1,66,4,3:20,3:20,
91,baseline_US12_page36.txt,12,36,3,1:5oh,1:5oh,1:5o
152,baseline_US19_page16.txt,19,16,1,00:10,00:10,
153,baseline_US19_page16.txt,19,16,2,03:00,03:00,
154,baseline_US19_page16.txt,19,16,3,00:30,00:30,
155,baseline_US19_page16.txt,19,16,4,00:05,00:05,
222,baseline_US33_page90.txt,33,90,4,1:3amin,1:39min,
267,baseline_US22_page51.txt,22,51,1,1:30h,1:30h,1:30
268,baseline_US22_page51.txt,22,51,2,1:h,1h,1
270,baseline_US22_page51.txt,22,51,4,1:30 h,1:30h,1:30
